# Import

In [1]:
import urllib.request
import datetime

# defs

In [13]:
def findUrlByDate(raw_html_data, year, month, day):
    target_string = "<a href=\'/art/" + str(year) + "/" + str(month) + "/" + str(day) + "/art"
    if raw_html_data.find(target_string)>-1:
        i1 = raw_html_data.index(target_string) # find target index
        i2 = raw_html_data[i1:].index("'")
        i3 = raw_html_data[i1+i2+1:].index("'")
        return "http://www.bjp.org.cn" + raw_html_data[i1+i2+1:i1+i2+i3+1]
    else:
        return "ERROR"

def trimHtmlMakeup(raw_html_data):
    trimed_data = raw_html_data
    i1 = trimed_data.find('<')
    i2 = trimed_data.find('>')
    while (i1>-1 and i2>-1):
        trimed_data = trimed_data[:i1] + trimed_data[i2+1:]
        i1 = trimed_data.find('<')
        i2 = trimed_data.find('>')
    return trimed_data

def findTitleAndIntro(raw_html_data):
    if raw_html_data.find("<title>")>-1:
        i1 = raw_html_data.index("<title>")
        i2 = raw_html_data.index("</title>")
        title = raw_html_data[i1+7:i2]
    else:
        i1 = raw_html_data.index("<strong>")
        i2 = raw_html_data.index("</strong>")
        title = raw_html_data[i1+8:i2]
    i3 = raw_html_data[i2+1:].find("<strong>说明：</strong>")
    i4 = raw_html_data[i2+1+i3:].find('</p><p style=\"text-align: center;\">')
    raw_intro = raw_html_data[i2+1+i3:i2+1+i3+i4]
    trimed_intro = trimHtmlMakeup(raw_intro)
    return title,trimed_intro

def findImageUrl(raw_html_data):
    i5 = raw_html_data.find("<a href=\"/picture/")
    if i5>-1:
        i6 = raw_html_data[i5:].index('"')
        i7 = raw_html_data[i5+i6+1:].index('"')
        return "http://www.bjp.org.cn" + raw_html_data[i5+i6+1:i5+i6+i7+1]
    else:
        return "ERROR"

# parameters

#### DAYS_TO_RETRIEVE: total days back from today to retrieve their pics

In [7]:
DAYS_TO_RETRIEVE = 30

# Main

### use "Cell -> Current Outputs -> Toggle Scrolling" to expand the output window

In [14]:
import ipywidgets as widgets
f = urllib.request.urlopen('http://www.bjp.org.cn/col/col89/index.html')
readIn = f.read().decode('utf-8')

yesterday = datetime.date.today() + datetime.timedelta(days=-1)
out_vbox_list = []
for i in range(DAYS_TO_RETRIEVE):
    date_inst = yesterday + datetime.timedelta(days=int(-1*i))
    solo_image_page_url = findUrlByDate(readIn, date_inst.year, date_inst.month, date_inst.day)
    if solo_image_page_url != "ERROR":
        solo_image_page = urllib.request.urlopen(solo_image_page_url)
        rrr = solo_image_page.read().decode('utf-8')
        title, intro = findTitleAndIntro(rrr)
        title_output = widgets.Output(layout={'border': '2px solid black'})
        with title_output:
            print(str(date_inst.year) + "-" + str(date_inst.month) + "-" + str(date_inst.day) + "  " + title)
        intro_output = widgets.Output()
        with intro_output:
            print(intro)
            print('\n'*3)
        jpeg_url = findImageUrl(rrr)
        if jpeg_url !="ERROR":
            image_url = urllib.request.urlopen(jpeg_url)
            image = image_url.read()
            pic = widgets.Image(value=image, format='jpeg')
        else:
            pic = widgets.Output()
            with pic:
                print(str(date_inst.year) + "-" + str(date_inst.month) + "-" + str(date_inst.day) + "  There is no image retrieved on this day.")
        out_vbox_list.append(title_output)
        out_vbox_list.append(pic)
        out_vbox_list.append(intro_output)
    else:
        error_output = widgets.Output(layout={'border': '2px solid black'})
        with error_output:
            print(str(date_inst.year) + "-" + str(date_inst.month) + "-" + str(date_inst.day) + "  There is no image retrieved on this day.")
        blank_output = widgets.Output()
        with blank_output:
            print('\n'*3)
        out_vbox_list.append(error_output)
        out_vbox_list.append(blank_output)
        
out_vbox = widgets.VBox(out_vbox_list)
display(out_vbox)

A Jupyter Widget